In [2]:
import pandas as pd
import csv
import importlib
import nltk
import collections
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re
import sys
import spacy
import string
from os import listdir
from os.path import isfile, join
import numpy as np
stopWords = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
data = pd.read_csv("April22-Supreme-Court-Confirmation-Hearing-Transcript.csv", encoding="ISO-8859-1")

In [19]:
data.columns

Index(['Total Order', 'Order', 'Year', 'Hearing', 'Title',
       'Speaker (Party)(or nominated by)', 'Speaker and title', 'Statements'],
      dtype='object')

In [30]:
len(list(data.Hearing.unique()))

15

In [32]:
data.columns

Index(['Total Order', 'Order', 'Year', 'Hearing', 'Title',
       'Speaker (Party)(or nominated by)', 'Speaker and title', 'Statements'],
      dtype='object')

In [34]:
data.Year.unique()

array([2017, 2010, 2009, 2006, 2005, 1994, 1993, 1991, 1990, 1987, 1986,
       1981, 1975, 1971])

In [52]:
data[:10]

,Total Order,Order,Year,Hearing,Title,Speaker (Party)(or nominated by),Speaker and title,Statements
0,1,1,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),"Chairman Grassley. Good morning, everybody. I ..."
1,2,2,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,Judge Gorsuch. Pleasure to be here. Thank you.
2,3,3,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),Chairman Grassley. This is a big day for you a...
3,4,4,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,"Judge Gorsuch. Just a little, Senator."
4,5,5,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),"Chairman Grassley. Yes. Before we begin, I wou..."
5,6,6,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,"Judge Gorsuch. Senator, this is quite a lot di..."
6,7,7,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),Chairman Grassley. Yes.
7,8,8,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,"Judge Gorsuch. I have got here somewhere, I am..."
8,9,9,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),"Chairman Grassley. Thank you. We thank you, Ju..."
9,10,10,2017,Neil M. Gorsuch,Senator,D,Senator Dianne Feinstein (CA),"Senator Feinstein. Thank you very much, Mr. Ch..."


In [301]:
hearings_names = list(data.Hearing.unique()) 
hearings_names

['Neil M. Gorsuch',
 'Elena Kagan',
 'Sonia Sotomayor',
 'Samuel A. Alito, Jr.',
 'John G. Roberts, Jr.',
 'Stephen G. Breyer',
 'Ruth Bader Ginsburg',
 'Clarence Thomas',
 'David H. Souter',
 'Anthony M. Kennedy',
 'William H. Rehnquist (to be chief)',
 'Antonin Scalia',
 "Sandra Day O'Connor",
 'John Paul Stevens',
 'William H. Rehnquist (to be Associate Justice) and Lewis F. Powell Jr.']

In [28]:
ls=[]
dict_of_hearings = {}
for i, row in data.iterrows():
    tup = (row['Year'], row['Hearing'])
    ls.append(tup)
for tup in ls:
    if tup not in dict_of_hearings:
        dict_of_hearings[tup]={}

In [37]:
list_of_tuples=[]
for i, row in data.iterrows():
    tup = (row['Speaker (Party)(or nominated by)'], row['Speaker and title'])
    if tup not in list_of_tuples:
        list_of_tuples.append(tup)
list_of_tuples

[('R', 'Senator Chuck Grassley (IA)'),
 ('R', 'Neil M. Gorsuch'),
 ('D', 'Senator Dianne Feinstein (CA)'),
 ('R', 'Senator Orrin Hatch (UT)'),
 ('D', 'Senator Patrick Leahy (VT)'),
 ('R', 'Senator John Cornyn (TX)'),
 ('D', 'Senator Dick Durbin (IL)'),
 ('R', 'Senator Mike Lee (UT)'),
 ('D', 'Senator Sheldon Whitehouse (RI)'),
 ('R', 'Senator Lindsey Graham (SC)'),
 ('D', 'Senator Amy Klobuchar (MN)'),
 ('R', 'Senator Ted Cruz (TX)'),
 ('D', 'Senator Al Franken (MN)'),
 ('R', 'Senator Ben Sasse (NE)'),
 ('D', 'Senator Chris Coons (DE)'),
 ('R', 'Senator Jeff Flake (AZ)'),
 ('D', 'Senator Richard Blumenthal (D-CT)'),
 ('R', 'Senator Mike Crapo (ID)'),
 ('R', 'Senator Thom Tillis (NC)'),
 ('D', 'Senator Mazie Hirono (HI)'),
 ('R', 'Senator John Neely Kennedy (LA)'),
 ('D', 'Senator Michael Bennet (CO)'),
 ('R', 'Senator Cory Gardner (CO)'),
 ('D', 'Neal Katyal'),
 ('R', 'Senator Jeff Sessions (AL)'),
 ('D', 'Senator Herb Kohl (WI)'),
 ('D', 'Senator Russ Feingold (WI)'),
 ('R', 'Senator 

In [30]:
for k, v in dict_of_hearings.items():
    for tuple_speaker in list_of_tuples:
        if tuple_speaker not in v:
            v[tuple_speaker]=[]

In [7]:
data.head()

,Total Order,Order,Year,Hearing,Title,Speaker (Party)(or nominated by),Speaker and title,Statements
0,1,1,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),"Chairman Grassley. Good morning, everybody. I ..."
1,2,2,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,Judge Gorsuch. Pleasure to be here. Thank you.
2,3,3,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),Chairman Grassley. This is a big day for you a...
3,4,4,2017,Neil M. Gorsuch,Nominee,R,Neil M. Gorsuch,"Judge Gorsuch. Just a little, Senator."
4,5,5,2017,Neil M. Gorsuch,Chairman,R,Senator Chuck Grassley (IA),"Chairman Grassley. Yes. Before we begin, I wou..."


In [339]:
partisan_dict={}
ls = []
for i, row in data.iterrows():
    if row['Speaker (Party)(or nominated by)']== "R" or row['Speaker (Party)(or nominated by)']== "D" :
        ls.append((row["Year"],row['Speaker (Party)(or nominated by)']))
for tup in ls:
    if tup not in partisan_dict:
        partisan_dict[tup]=[]


In [340]:
for i, row in data.iterrows():
    for key, val in partisan_dict.items():
        if row['Year'] == key[0] and row['Speaker (Party)(or nominated by)'] == key[1] and row['Hearing']!=row['Speaker and title']:
            val.append(row['Statements'])

In [303]:
partisan_dict.keys()

dict_keys([(2017, 'R'), (2017, 'D'), (2010, 'D'), (2010, 'R'), (2009, 'D'), (2009, 'R'), (2006, 'R'), (2006, 'D'), (2005, 'R'), (2005, 'D'), (1994, 'D'), (1994, 'R'), (1993, 'D'), (1993, 'R'), (1991, 'D'), (1991, 'R'), (1990, 'D'), (1990, 'R'), (1987, 'D'), (1987, 'R'), (1986, 'R'), (1986, 'D'), (1981, 'R'), (1981, 'D'), (1975, 'D'), (1975, 'R'), (1971, 'D'), (1971, 'R')])

In [333]:
#partisan_dict[(2017, 'R')]

In [341]:
republican_count = {}
democrat_count={}
keys_to_keep = []
for key,val in partisan_dict.items():
    if key[1]=="R":
        republican_count[key] = val
    else:
        democrat_count[key] = val


In [342]:
republican_count = clean_words(republican_count)
democrat_count = clean_words(democrat_count)
republican_count= total_word_count(republican_count)
democrat_count = total_word_count(democrat_count)

In [304]:
list_of_hearings=[]
for i, row in data.iterrows():
    tup = (row['Hearing'], row['Year'])
    list_of_hearings.append(tup)
nominees_year = set(list_of_hearings)
nominees_year

{('Anthony M. Kennedy', 1987),
 ('Antonin Scalia', 1986),
 ('Clarence Thomas', 1991),
 ('David H. Souter', 1990),
 ('Elena Kagan', 2010),
 ('John G. Roberts, Jr.', 2005),
 ('John Paul Stevens', 1975),
 ('Neil M. Gorsuch', 2017),
 ('Ruth Bader Ginsburg', 1993),
 ('Samuel A. Alito, Jr.', 2006),
 ("Sandra Day O'Connor", 1981),
 ('Sonia Sotomayor', 2009),
 ('Stephen G. Breyer', 1994),
 ('William H. Rehnquist (to be Associate Justice) and Lewis F. Powell Jr.',
  1971),
 ('William H. Rehnquist (to be chief)', 1986)}

In [136]:
list_of_hearings=[]
for i, row in data.iterrows():
    if row['Hearing'] == row['Speaker and title']:
        tup = (row['Hearing'], row['Year'],row['Speaker (Party)(or nominated by)'] )
    
    list_of_hearings.append(tup)
nominees_year_party = set(list_of_hearings)
len(nominees_year_party)
#republicans = ["Sandra Day O'Connor",'Anthony M. Kennedy', 'Antonin Scalia', ]

13

In [132]:
nominate_dict = {}
for nominate in list(nominees_year_party):
    if nominate not in nominate_dict:
        nominate_dict[nominate]=[]

In [123]:
# nominate_dict = {}
# for nominate in list(nominees_year):
#     if nominate not in nominate_dict:
#         nominate_dict[nominate]=[]

In [133]:
for key, val in nominate_dict.items():
    for i, row in data.iterrows():
        if row['Hearing']==row['Speaker and title']==key[0]:
            val.append(row['Statements'])        

In [196]:
names =[]
for i, row in data.iterrows():
        name = row['Hearing']
        names.append(name)
names=list(set(names))

In [199]:
names

['William H. Rehnquist (to be chief)',
 'Stephen G. Breyer',
 "Sandra Day O'Connor",
 'David H. Souter',
 'Neil M. Gorsuch',
 'Ruth Bader Ginsburg',
 'John G. Roberts, Jr.',
 'John Paul Stevens',
 'Samuel A. Alito, Jr.',
 'William H. Rehnquist (to be Associate Justice) and Lewis F. Powell Jr.',
 'Antonin Scalia',
 'Sonia Sotomayor',
 'Anthony M. Kennedy',
 'Elena Kagan',
 'Clarence Thomas']

In [203]:
last_names = []
for index,name in enumerate(names):
    one_name = name.split()
    if index==0:
        last_names.append('rehnquist')
    elif index==6:
        last_names.append('roberts')
    elif index==8:
        last_names.append('alito')
    elif index==9:
        last_names.append('powell')  
    else:
        last_names.append(one_name[-1].lower())


In [204]:
last_names

['rehnquist',
 'breyer',
 "o'connor",
 'souter',
 'gorsuch',
 'ginsburg',
 'roberts',
 'stevens',
 'alito',
 'powell',
 'scalia',
 'sotomayor',
 'kennedy',
 'kagan',
 'thomas']

In [330]:
list_to_delete = ['senator', 'judge', 'chairman', 'leahy', 'grassley'] + last_names


In [140]:
nominate_dict.keys()

dict_keys([('John Paul Stevens', 1975, 'R'), ('Samuel A. Alito, Jr.', 2006, 'R'), ('Elena Kagan', 2010, 'D'), ('Sonia Sotomayor', 2009, 'D'), ('Ruth Bader Ginsburg', 1993, 'D'), ('Neil M. Gorsuch', 2017, 'R'), ('Stephen G. Breyer', 1994, 'D'), ("Sandra Day O'Connor", 1981, 'R'), ('Anthony M. Kennedy', 1987, 'R'), ('John G. Roberts, Jr.', 2005, 'R'), ('David H. Souter', 1990, 'R'), ('Antonin Scalia', 1986, 'R'), ('Clarence Thomas', 1991, 'R')])

In [227]:
def clean_words(speech_dict):
    """
    Creates a dictionary which keys are the
    tuple (nominee,year, party), or (year, party), and its value a list of
    tokenized words without StopWords.
    
    Inputs: Dictionary with speeches
    Returns: Dictionary
    """
    clean_dict = {}
    tokenizer = RegexpTokenizer(r'\w+')
    stopWords = set(stopwords.words('english'))
    for k, v in speech_dict.items():
        if k not in clean_dict:
            clean_dict[k]= []
        for lines in v:
            lines = str(lines).lower().translate(string.punctuation)
            lines=tokenizer.tokenize(lines)
            for line in lines:
                if line not in stopWords and line not in set(list_to_delete):
                    clean_dict[k].append([line])
                
    clean_dict = {k: [val for sublist in v for val in sublist] for k,v in clean_dict.items()}
    return clean_dict

In [331]:
nom = clean_words(nominate_dict)

In [332]:
ls_counts = []
for key in nominate_dict.keys():
    ls_counts.append((key[1],len(nom[key]), key[2]))
dataframe_length_nominees= pd.DataFrame(ls_counts, columns=['year', 'length_nominee', 'party_nominee'])

In [296]:
dataframe_length_nominees.sort_values(by=['year'])

,year,length_nominee,party_nominee
0,1975,8335,R
7,1981,17321,R
11,1986,8110,R
8,1987,16033,R
10,1990,29770,R
12,1991,28132,R
4,1993,26000,D
6,1994,24275,D
9,2005,32686,R
1,2006,28305,R


In [187]:
nominate_dict.keys()

dict_keys([('John Paul Stevens', 1975, 'R'), ('Samuel A. Alito, Jr.', 2006, 'R'), ('Elena Kagan', 2010, 'D'), ('Sonia Sotomayor', 2009, 'D'), ('Ruth Bader Ginsburg', 1993, 'D'), ('Neil M. Gorsuch', 2017, 'R'), ('Stephen G. Breyer', 1994, 'D'), ("Sandra Day O'Connor", 1981, 'R'), ('Anthony M. Kennedy', 1987, 'R'), ('John G. Roberts, Jr.', 2005, 'R'), ('David H. Souter', 1990, 'R'), ('Antonin Scalia', 1986, 'R'), ('Clarence Thomas', 1991, 'R')])

In [190]:
years_republican=[]
years_democrat=[]
for key in nominate_dict.keys():
    if key[2]=="R":
        years_republican.append(key[1])
years_republican

[1975, 2006, 2017, 1981, 1987, 2005, 1990, 1986, 1991]

In [186]:
partisan_dict.keys()

dict_keys([(2017, 'R'), (2017, 'D'), (2010, 'D'), (2010, 'R'), (2009, 'D'), (2009, 'R'), (2006, 'R'), (2006, 'D'), (2005, 'R'), (2005, 'D'), (1994, 'D'), (1994, 'R'), (1993, 'D'), (1993, 'R'), (1991, 'D'), (1991, 'R'), (1990, 'D'), (1990, 'R'), (1987, 'D'), (1987, 'R'), (1986, 'R'), (1986, 'D'), (1981, 'R'), (1981, 'D'), (1975, 'D'), (1975, 'R'), (1971, 'D'), (1971, 'R')])

In [261]:
partisan = clean_words(partisan_dict)

In [277]:
## REPUBLICANS QUESTIONING A DEMOCRAT CANDIDATE
rep_vs_dem = {}
dem_vs_rep = {}
rep_vs_rep = {}
dem_vs_dem = {}

for k, val in partisan.items():
    if k[0] not in years_republican and k[1]=='R':
        rep_vs_dem[k[0]] = val

## DEMOCRATS QUESTIONING A REPUBLICAN CANDIDATE
for k, val in partisan.items():
    if k[0] in years_republican and k[1]=='D':
        dem_vs_rep[k[0]] = val

## REPUBLICANS QUESTIONING A REPUBLICAN CANDIDATE
for k, val in partisan.items():
    if k[0] in years_republican and k[1]=='R':
        rep_vs_rep[k[0]] = val

## DEMOCRATS QUESTIONING A DEMOCRAT CANDIDATE
for k, val in partisan.items():
    if k[0] not in years_republican and k[1]=='D':
        dem_vs_dem[k[0]] = val

In [278]:
def df_length(dictionary, party):
    
    ls_counts = []
    for year,text in dictionary.items():
        ls_counts.append((year,len(text)))
    dataframe_length= pd.DataFrame(ls_counts, columns=['year', str('length'+"_"+ party)])
    return dataframe_length

In [281]:
length_rep_vs_dem = df_length(rep_vs_dem, "rep")
length_rep_vs_rep = df_length(rep_vs_rep, "rep")
length_dem_vs_rep = df_length(dem_vs_rep, "dem")
length_dem_vs_dem = df_length(dem_vs_dem, "dem")

In [288]:
years_republican

[1975, 2006, 2017, 1981, 1987, 2005, 1990, 1986, 1991]

In [286]:
length_rep = length_rep_vs_dem.append(length_rep_vs_rep)
length_dem = length_dem_vs_dem.append(length_dem_vs_rep)


In [297]:
lengths_r_d= pd.merge(length_rep, length_dem, on='year')
lengths = pd.merge(lengths_r_d, dataframe_length_nominees, on ='year')

In [299]:
lengths.sort_values(by=['year'])

,year,length_rep,length_dem,length_nominee,party_nominee
12,1975,1451,5457,8335,R
11,1981,15108,11535,17321,R
10,1986,41355,29206,8110,R
9,1987,17103,17153,16033,R
8,1990,17239,26177,29770,R
7,1991,29746,39965,28132,R
3,1993,21987,27903,26000,D
2,1994,17843,25427,24275,D
6,2005,26059,26054,32686,R
5,2006,33173,30903,28305,R


In [219]:
def total_word_count(data):
    '''
    Function to create a dictionary of words counts by president and year. A larger function that allows the user to choose the data cut calls this function.
    
    inputs: 
        data: speech text dictionart
    output:
       dict_use: a dictionary of dictionary of counts.
    '''
    dict_use = {}
    for k in data.keys():
        for word in data[k]:
            if word in dict_use.keys():
                        dict_use[word] += 1
            else:
                dict_use[word] = 1  
    return dict_use


In [231]:
dict_test = total_word_count(nom)

In [308]:
def df_of_counts(dictionary, k_words):
    top_list = sorted(dictionary, key = dictionary.get, reverse=True)[:k_words]
    top_words = {word:val for word,val in dictionary.items() if word in top_list}
    dataframe_with_counts= pd.DataFrame(top_words.items(), columns=['Words', 'Count'])
    return dataframe_with_counts

In [343]:
counts_nominates = df_of_counts(dict_test,40).sort_values(by=['Count'], ascending=False)
counts_democrats = df_of_counts(democrat_count,40).sort_values(by=['Count'], ascending=False)
counts_republicans = df_of_counts(republican_count,40).sort_values(by=['Count'], ascending=False)

In [344]:
counts_nominates
counts_nominates.to_csv("nominates.csv",index=False)

In [345]:
counts_democrats
counts_democrats.to_csv('democrats.csv',index=False)

In [346]:
counts_republicans
counts_republicans.to_csv('republicans.csv', index=False)

In [347]:
lengths.to_csv('lengths.csv', index=False)